# Poor Confidence Analysis
This analysis aims to improve the NER model by studying the predictions done by the model. This is done firstly by identifying the weakness or limitations discovered from the prediction results. So, by looking at the outputs, the confidence score can be the area of focus for this analysis.

The idea of this analysis is to find out if the low confidence scores are justifiable. For example, is the confidence score related to the volume of the dataset, frequency of the labels, or word rarity? This analysis covers the:
1. label frequency
2. text frequency
3. confidence score distribution
4. text length
5. text complexity

This can be determined by answering these questions:
1. Is there a correlation between label frequency and confidence scores?    (`label`)
2. Are low-confidence predictions associated with text frequency/rarity?    (`text`)
3. Do certain label types show systematic confidence patterns?              (`label`)
4. Is confidence score correlated with text length?                         (`text`)
5. Is confidence score correlated with complexity?                          (`text`)

Based on these questions, 

* If low scores correlate with rare labels: Augment training data for underrepresented classes.
* If low scores correlate with rare words: Add domain-specific vocabulary for training.
* If low scores are consistent with certain label group: Focused annotation to cator specific labelings.

If all are unclear: Review model architecture or hyperparameter tuning

-------------
------
## Data Pre-processing

#### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats             # correlation

#### Load Dataset

In [12]:
# load csv
df = pd.read_csv('ner_news_malay\model_gliner\data\\results_main.csv')
df.head()

<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Arif Irfan Ibrahim\AppData\Local\Temp\ipykernel_12440\252216309.py:2: SyntaxWarning: invalid escape sequence '\m'
  df = pd.read_csv('ner_news_malay\model_gliner\data\\results_main.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'ner_news_malay\\model_gliner\\data\\results_main.csv'

In [ ]:
df.info

In [ ]:
# normalize text
df['text'] = df['text'].str.lower()
df.head()

In [ ]:
# duplicates
df.drop_duplicates().head()

-------------
--------
## Exploratory Data Analysis
This part aims to explore the dataset in hopes to uncover hidden perspectives.

#### Label Distribution

In [ ]:
df.head()

In [ ]:
# label distribution
df['label'].value_counts()

In [ ]:
# visualize
label_freq = df['label'].value_counts().index

plt.figure(figsize=(18,5))
sns.countplot(x=df['label'],data=df, order=label_freq)
plt.xticks(rotation=45)
plt.show()

From this graph, it is safe to assume that the high-frequency labels are the 3 highest, while the others are teh low-frequency labels. This is because the gap between the third and the fourth highest is relatively big (1500 vs 500), when compared to other inter-label gaps.

#### High and Low Frequency Labels

In [ ]:
# overview
df.head()

In [ ]:
# drop columns
freq = df.drop(['start','end','text'], axis=1)
freq.head()

In [ ]:
# list labels
freq['label'].unique()

In [ ]:
# split into higher and lower frequency groups
high_freq_label = ['PERSON','ORG','LOC']
low_freq_label = ['PRODUCT', 'QUANTITY', 'EVENT', 'GPE',
       'CARDINAL', 'TIME', 'LAW', 'MONEY', 'PERCENT', 'WORK_OF_ART',
       'NORP', 'FAC', 'ORDINAL']

freq_high = freq[freq['label'].isin(high_freq_label)]
freq_low = freq[freq['label'].isin(low_freq_label)]

#### Confidence Score Disctribution

In [ ]:
# get mean confidence score
conf = df.groupby('label')['score'].mean().reset_index()
conf = conf.sort_values('score', ascending=False)

In [ ]:
# visualize
fig = plt.figure(figsize=(20, 8))
ax = fig.add_subplot(1, 1, 1) 
bars = ax.bar(conf['label'], conf['score'], color='purple')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2,
            height + 0.01, f'{height:.3f}',
            ha='center', va='bottom', fontsize=10)

ax.set_title('Confidence Score Distribution')
ax.set_xlabel('Entity Label')
ax.set_ylabel('Average Confidence Score')
ax.set_ylim(0.5, 0.9)
ax.axhline(y=0.7,linestyle='--')
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()


Based on this distribution, there are 5 labels that are considered to be low-confidence.

#### Confidence Scores in High and Low Frequency Labels

In [ ]:
# get mean score
label_score_mean_high = freq_high.groupby('label')['score'].mean().reset_index()
label_score_mean_low = freq_low.groupby('label')['score'].mean().reset_index()

In [ ]:
# sort mean score
label_score_mean_high = label_score_mean_high.sort_values('score', ascending=False)
label_score_mean_low = label_score_mean_low.sort_values('score', ascending=False)

In [ ]:
# visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# high frequency labels
bars1 = ax1.bar(label_score_mean_high['label'], 
               label_score_mean_high['score'],
               color='green')

# labeling
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height + 0.01,
            f'{height:.3f}', ha='center', va='bottom', fontsize=10)

# setting
ax1.set_title('High Frequency Entities')
ax1.set_xlabel('Entity Label')
ax1.set_ylabel('Average Confidence Score')
ax1.set_ylim(0.5, 0.9)
ax1.tick_params(axis='x', rotation=45)

# low frequency labels
bars2 = ax2.bar(label_score_mean_low['label'], 
               label_score_mean_low['score'],
               color='teal')

# labeling
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height + 0.01,
            f'{height:.3f}', ha='center', va='bottom', fontsize=10)

# setting
ax2.set_title('Low Frequency Entities')
ax2.set_xlabel('Entity Label')
ax2.set_ylabel('Average Confidence Score')
ax2.set_ylim(0.5, 0.9)
ax2.tick_params(axis='x', rotation=45)

# display
plt.tight_layout
plt.show()

Based on these graphs, the lowest confidence score for the high-frequency labels is around 76%, while the highest confidence score for the low-frequency labels is around 82%. This means that even if the label is relatively rare, the confidence score can still be high. Even so, it cannot be a certain that there is no correlation between the label rarity and the confidence score. Therefore, a correlation analysis can be done to further confirm this.

#### Low Rarity Texts

In [ ]:
# overview
df.head()

In [ ]:
# text frequency
text_freq = df['text'].value_counts().reset_index() 
text_freq

In [ ]:
# texts that appeared only once
text_low = text_freq[text_freq['count'] == 1]
text_low

From this, there are about 4000 texts that appeared only once. That's more than half of the whole dataset (6000 texts)! This is also while excluding the texts that appeared only twice or thrice.

However, this is to be expected since texts with unique nomenclature or numbers might be a factor to this phenomenon. 

#### Distribution of Confidence Score in Low Rarity Texts

In [ ]:
# overview
df.head()

In [ ]:
# get score for low rarity texts
text_conf = df.groupby('text')['score'].mean().reset_index()  # get mean confidence score

text_low_conf = pd.merge(text_freq, text_conf, on='text')
text_low_conf = text_low_conf[text_low_conf['count'] == 1]
text_low_conf.sort_values('score',ascending=False)

Here, we can see that the rare texts has the confidence score at both high and low ends.

In [ ]:
score_low = df[df['score'] < 0.7]
score_low.sort_values('score',ascending=False)

There are about 5000 texts that has low confidence scores.

#### Distribution of Text Lengths

In [ ]:
txt_len = df.copy()
txt_len['chara'] = df['text'].str.len()
txt_len

In [ ]:
# distribution of text length
plt.figure(figsize=(14, 7))

# histogram
sns.histplot(txt_len['chara'], bins=50, kde=True, color='teal')

# mean line
mean_length = txt_len['chara'].mean()
plt.axvline(mean_length, color='red', linestyle='--', 
            label=f'Mean: {mean_length:.0f} chars')

# median line
median_length = txt_len['chara'].median()
plt.axvline(median_length, color='blue', linestyle='--', 
            label=f'Median: {median_length:.0f} chars')

# plot
plt.title('Distribution of Text Lengths')
plt.xlabel('Number of Characters')
plt.ylabel('Frequency')
plt.legend()
plt.grid(alpha=0.3)

plt.show()

This distribution histogram is right-skewed. It comprises of mostly short texts.

--------------
--------------
## Data Analysis

-------------
### Q1: Is there a correlation between label frequency and confidence scores?
Study on the mean confidence scores and label frequency correlation. This is to see if high frequency means high confidence score, or vice versa.

#### Data Preparation

In [ ]:
# overview
df.head()

In [ ]:
# create new dataset
q1_label = df['label'].value_counts().reset_index()         # get label frequency
q1_conf = df.groupby('label')['score'].mean().reset_index() # get mean confidence score

q1_label_analysis = pd.merge(q1_label, q1_conf, on='label')
q1_label_analysis.sort_values('score',ascending=False)

#### Correlation Analysis
In this analysis, the tests that will be conducted are:
1. Pearson correlation - to determine the linear relationship
2. Spearman correlation - to determine the monotonic relationship
3. Shapiro-Wilk test - to determine if its normally distributed

**Null hypothesis:**\
There is no correlation between label frequency and confidence scores.

**Alternative hypothesis:**\
There is a correlation between label frequency and confidence scores.

In [ ]:
# pearson
q1_pearson_corr, q1_p_value = stats.pearsonr(
    q1_label_analysis['count'],
    q1_label_analysis['score']
)

# spearman
q1_spearman_rho, q1_p_spearman = stats.spearmanr(
    q1_label_analysis['count'],
    q1_label_analysis['score']
)

# normality test
q1_shapiro_test = stats.shapiro(q1_label_analysis['score'])

In [ ]:
print(f"Spearman rho    : {q1_spearman_rho:.3f}")
print(f"Spearman p      : {q1_p_spearman:.4f}\n")
print(f"Pearson r       : {q1_pearson_corr:.3f}")
print(f"Pearson p       : {q1_p_value:.4f}\n")
print(f"Shapiro-Wilk W  : {q1_shapiro_test[0]:.3f}")
print(f"Shapiro-Wilk p  : {q1_shapiro_test[1]:.4f}\n")

Based on these results:
* There is a significant (p < 0.05) strong positive monotonic relationship (rho > 0.6). This means that as label frequency increases, confidence scores consistently increase, but not necessarily in a straight-line pattern - Spearman correlation test
* There is a significant (p < 0.05) moderate positive linear relationship (r > 0.4, r < 0.6). This means that higher frequency moderately predicts higher confidence in a linear pattern. - Pearson correlation test
* Since the rho > r, the monotonic relationship is stronger than the linear relationship. This means that even though higher frequency boosts the confidence score, the boost will decrease with higher frequency.
* Since both of these tests are significant, the results are not random.
* The data is normally distributed (p > 0.05). - Shapiro-Wilk test

Therefore, we reject the null hypothesis.

For fine-tuning, training data that focuses on the low-frequency labels can be used for augmentation.

In [ ]:
# filter low-frequency labels in main dataset
ft_q1 = df[df['label'].isin(low_freq_label)]
ft_q1 = ft_q1[ft_q1['score'] < 0.7]
ft_q1.sort_values('score')

------------
### Q2: Are low-confidence predictions correlated with text frequency/rarity?
Study on the low confidence scores and text frequency. This is to see if lower rarity means lower confidence score, or vice versa.

#### Data Preparation

In [ ]:
# overview
df.head()

In [ ]:
# create new dataset
q2_text = df['text'].value_counts().reset_index()           # get text frequency
q2_conf = df.groupby('text')['score'].mean().reset_index()  # get mean confidence score

q2_text_analysis = pd.merge(q2_text, q2_conf, on='text')
q2_text_analysis

#### Correlation Analysis
In this analysis, the tests that will be conducted are:
1. Pearson correlation - to determine the linear relationship
2. Spearman correlation - to determine the monotonic relationship

**Null hypothesis:**\
There is no correlation between text frequency and confidence scores.

**Alternative hypothesis:**\
There is a correlation between text frequency and confidence scores.

In [ ]:
# pearson
q2_pearson_corr, q2_p_value = stats.pearsonr(
    q2_text_analysis['count'],
    q2_text_analysis['score']
)

# spearman
q2_spearman_rho, q2_p_spearman = stats.spearmanr(
    q2_text_analysis['count'],
    q2_text_analysis['score']
)

In [ ]:
print(f"Spearman rho: {q2_spearman_rho:.3f}")
print(f"Spearman p: {q2_p_spearman:.4f}\n")
print(f"Pearson r: {q2_pearson_corr:.3f}")
print(f"Pearson p: {q2_p_value:.4f}\n")

Based on these results:
* There is a significant (p < 0.05) but negligible monotonic relationship (rho < 0.1). This means that there is no meaningful relationship between text and confidence score. - Spearman correlation test
* There is no significant (p > 0.05) linear relationship (r < 0.1). This means that there is no evidence of linear relationship between text and confidence score. - Pearson correlation test

Therefore, we do not reject the null hypothesis.

The fine-tuning should be aimed at other factors instead.

This further justifies the EDA,  where the confidence scores are covered from both high and low ends from the texts, despite the rarity of the texts.

------------
### Q3: Do certain label types show systematic confidence patterns?
Analyze low confidence score distributions per label type. This is to see whether certain labels are under-trained.

#### Data Preparation

In [ ]:
# overview
df.head()

In [ ]:
# sort and filter score < 0.7
q3 = df.sort_values('score', ascending=False)
q3 = q3[q3['score'] < 0.7]
q3

In [ ]:
# number of labels
print(f"number of labels with low-scores    : {len(q3['label'].unique())}")
print(f"number of labels on main            : {len(df['label'].unique())}")

It seems that there are low-confidence scores for all  types of lables. However, based on our EDA, the lables that has low-average confidence scores are the FAC, NORP, QUANTITY, CARDINAL, and ORDINAL (5 labels). This might be because of outliers in each label type. But to be sure, the weight of the low confidence scores and high confidence scores can be measured for each label. Since the number of the scores cannot be compared because of relative frequencies, percentages can be used for the comparison. 

#### Compare Confidence Score of Individual Labels
Get horizontal bar chart for each label (16 bar charts), with confidence score of 0.7 as the sperator for each chart.

In [ ]:
# get back full dataset
q3 = df
q3.head()

In [ ]:
# get number labels of high and low confidence
q3_conf = q3.groupby('label')['score'].agg(
    high_conf=lambda x: (x >= 0.7).sum(),
    low_conf=lambda x: (x < 0.7).sum(),
    total='count'
).reset_index()
q3_conf

In [ ]:
# get percentages
q3_conf['high_conf_pct'] = (q3_conf['high_conf'] / q3_conf['total'] * 100).round(1)
q3_conf['low_conf_pct'] = (q3_conf['low_conf'] / q3_conf['total'] * 100).round(1)
q3_conf

In [ ]:
# Sort by high confidence percentage (descending)
q3_conf = q3_conf.sort_values('high_conf_pct', ascending=False)
q3_conf

In [ ]:
# visualize
plt.figure(figsize=(14, 10))
y_pos = np.arange(len(q3_conf))
bar_height = 0.7

# stacked horizontal bar charts
low_bars = plt.barh(y_pos, q3_conf['low_conf_pct'], 
                    height=bar_height, color='brown', label='Confidence < 0.7')
high_bars = plt.barh(y_pos, q3_conf['high_conf_pct'], 
                     left=q3_conf['low_conf_pct'], 
                     height=bar_height, color='teal', label='Confidence >= 0.7')

# annotations
for i, (low_pct, high_pct, total) in enumerate(zip(q3_conf['low_conf_pct'], 
                                                   q3_conf['high_conf_pct'], 
                                                   q3_conf['total'])):
    # low confidence scores
    if low_pct > 1:     # to ensure space
        plt.text(low_pct/2, i, 
                 f'{low_pct:.1f}%\n({int(q3_conf["low_conf"].iloc[i])})', 
                 ha='center', va='center', color='white', fontsize=9, fontweight='bold')
    
    # high confidence scores
    if high_pct > 1:    # to ensure space 
        plt.text(q3_conf['low_conf_pct'].iloc[i] + high_pct/2, i, 
                 f'{high_pct:.1f}%\n({int(q3_conf["high_conf"].iloc[i])})', 
                 ha='center', va='center', color='white', fontsize=9, fontweight='bold')
    
    # total
    plt.text(105, i, f'n={total}')

    # line at 50%

# plot
plt.axvline(x=50, color='grey', linestyle='--')
plt.gca().invert_yaxis()
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.12), ncol=2)
plt.yticks(y_pos, q3_conf['label'])
plt.title('Confidence Distribution by Label Type')
plt.xlim(0, 110)
plt.tight_layout()
plt.show()

There are a few key points that can be taken away from this chart, but to break it down further:

Based on >= 50% high confidence score coverage(10 labels):
* Even though 'MONEY','PERCENT', 'GPE', 'LAW', 'TIME', 'PRODUCT', and 'EVENT' are considered to be low-frequency labels, based on the EDA, they have high confidence score coverage.
* The high-frequency labels, 'ORG', 'PERSON', and 'LOC' all have high confidence score coverage.
* The 'PERCENT' may be an outlier, because despite having the lowest number of labels in this category, it is has the second highest high confidence score coverage. This might be because of the simplicity/less variability of the annotation i.e. the symbol '%' is used as the indication for the label.

Based on < 50% high confidence score coverage(6 labels):
* All of the labels in these category are the low-frequency labels based on the EDA.
* The ' QUANTITY' label may in an outlier, because despite having the highest number of labels, about double from the second highest, it is has a poor high confidence score coverage. This might be because of the variability of the annotation, from being numbers.

From this, a label-focused annotation can be done for the fine-tuning:
* 'ORDINAL', 'CARDINAL', 'QUANTITY', 'NORP', 'WORK_OF_ART', and 'EVENT' should be the labels of focus.

In [ ]:
# get low high confidence covergage labels with low confidence
low_high_cov_label = ['ORDINAL', 'CARDINAL', 'QUANTITY', 'NORP', 'WORK_OF_ART', 'EVENT']

ft_q3 = df[df['label'].isin(low_high_cov_label)]
ft_q3 = ft_q3[ft_q3['score'] < 0.7]
ft_q3.sort_values('score')

-----------
### Q4: Is confidence score correlated with text length or complexity?
Find the correlation between length of texts and confidence scores.

#### Data Preparation

In [ ]:
# overview
df.head()

In [ ]:
# get characters
q4_chara_analysis = txt_len.copy()
q4_chara_analysis

#### Correlation Analysis
In this analysis, the tests that will be conducted are:
1. Pearson correlation - to determine the linear relationship
2. Spearman correlation - to determine the monotonic relationship

**Null hypothesis:**\
There is no correlation between text length and confidence scores.

**Alternative hypothesis:**\
There is a correlation between text length and confidence scores.

In [ ]:
# pearson
q4_pearson_corr, q4_p_value = stats.pearsonr(
    q4_chara_analysis['chara'],
    q4_chara_analysis['score']
)

# spearman
q4_spearman_rho, q4_p_spearman = stats.spearmanr(
    q4_chara_analysis['chara'],
    q4_chara_analysis['score']
)

In [ ]:
# display
print(f"Spearman rho: {q4_spearman_rho:.3f}")
print(f"Spearman p: {q4_p_spearman:.4f}\n")
print(f"Pearson r: {q4_pearson_corr:.3f}")
print(f"Pearson p: {q4_p_value:.4f}\n")

Based on these readings,

There is a highly significant (p < 0.05) weak positive monotonic and linear relationship (rho < 0.4, r < 0.4). This means that the effects of the text length to confidence score is negligible.

Therefore, we reject the null hypothesis. However, the relationship has no practical importance.

So, no fine-tuning can be uased by the text lengths.

-----------
### Q5: Is confidence score correlated with text complexity?
Find the correlation between the complexity of texts and confidence scores.

#### Data Preparation

In [ ]:
# overview
df.head()

In [ ]:
# feature selection
q5_complex_analysis = df[['text', 'score']].copy()
q5_complex_analysis

In [ ]:
# text processing
text = q5_complex_analysis['text']
words = text.str.split()
word_count = words.str.len()
word_count.sort_values()

In [ ]:
# word diversity
word_diversity = words.apply(set).str.len() / word_count
word_diversity.sort_values()

In [ ]:
# average word length
avg_word_length = words.str.join(' ').str.len() / word_count
avg_word_length.sort_values()

In [ ]:
# special character ratio
special_char_ratio = text.str.count(r'[^\w\s]') / text.str.len()
special_char_ratio.sort_values()

In [ ]:
# average complexity
q5_complex_analysis = q5_complex_analysis.assign(
    complexity=(
        word_diversity + 
        avg_word_length / 10 + 
        special_char_ratio
    ) / 3  
)

In [ ]:
# display
q5_complexity = q5_complex_analysis[['text', 'score', 'complexity']].sort_values('complexity')
q5_complexity.sort_values('complexity')

#### Correlation Analysis
In this analysis, the tests that will be conducted are:
1. Pearson correlation - to determine the linear relationship
2. Spearman correlation - to determine the monotonic relationship

**Null hypothesis:**\
There is no correlation between text complexity and confidence scores.

**Alternative hypothesis:**\
There is a correlation between text complexity and confidence scores.

In [ ]:
# pearson
q5_pearson_corr, q5_p_value = stats.pearsonr(
    q5_complex_analysis['complexity'],
    q5_complex_analysis['score']
)

# spearman
q5_spearman_rho, q5_p_spearman = stats.spearmanr(
    q5_complex_analysis['complexity'],
    q5_complex_analysis['score']
)

In [ ]:
# display
print(f"Spearman rho: {q5_spearman_rho:.3f}")
print(f"Spearman p: {q5_p_spearman:.4f}\n")
print(f"Pearson r: {q5_pearson_corr:.3f}")
print(f"Pearson p: {q5_p_value:.4f}\n")

From this: 
* There is a not significant (p > 0.05) positive monotonic relationship (rho < 0.4). - Spearman correlation test
* There is a significant (p < 0.05) positive weak linear relationship (r < 0.4). - Pearson correlation test